# Extract regions of a chromosome using qctool

In [ ]:
[global]
# Work directory where output will be saved to
parameter: cwd = path
# Region specifications
parameter: region_file = path
# Path to bgen/exome files
parameter: genoFile = paths
# Number of tasks to run in each job on cluster
parameter: job_size = int
# Number of threads
parameter: numThreads = 8
# Specify the container to use
#parameter: container_lmm = 'statisticalgenetics/lmm:3.0'
parameter: qctool_module = '''
module load Qctool/2.0.8
echo "Module qctool loaded"
{cmd}
'''
fail_if(not region_file.is_file(), msg = 'Cannot find regions to extract. Please specify them using ``--region-file`` option.')
# Load all regions of interest. Each item in the list will be a region: (chr, start, end)
regions = list(set([tuple(x.strip().split()) for x in open(region_file).readlines() if x.strip()]))

In [ ]:
# Filter bgen files chr21 and chr22 with only the 100 individuals
# Cluster script mwe_chr2.sh
[qctool_1]
# Output the bgen file with 8bit formatting
parameter: bgen_bits=8
input: genoFile, for_each='regions', group_by='1'
output: f'{cwd}/{_input:bn}.{_regions[0]}_{_regions[1]}_{_regions[2]}.bgen'
task: trunk_workers = 1, trunk_size = job_size, cores = numThreads, walltime = '48h', mem = '60G', tags = f'{step_name}_{_output:bn}'
bash: expand= "${ }", workdir = cwd, stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', template = qctool_module
    qctool \
    -g ${_input} \
    -og ${_output} \
    -incl-range ${regions[0]}:${regions[1]}-${regions[2]} \
    -bgen-bits ${bgen_bits} \
    -threads ${numThreads}

In [ ]:
[qctool_2]
input: output_from=('qctool')
output:f'{cwd}/{_input:bn}.{regions}.bgen.bgi'
bash: expand= "${ }", workdir = cwd, stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    bgenix \
    -g ${_input} \
    -index

In [3]:
sos dryrun ~/project/UKBB_GWAS_dev/workflow/qctool_extract_region.ipynb qctool \
    --cwd ~/test \
    --region_file ~/test/region_extract/test_chr5_73072354_73144845 \
    --genoFile /home/dmc2245/test/region_extract/ukb_imp_5_v3_chr5_73072354_73144845.bgen

ERROR: Unrecognized command line option /home/dmc2245/test
bash: --region_file: command not found
bash: --genoFile: command not found


: 127